In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
import re


In [2]:
options = Options()
options.add_argument("--start-maximized")  # Abrir el navegador maximizado

# Crear una instancia del navegador usando el WebDriver Manager incorporado
driver = webdriver.Chrome(options=options)
driver.get('https://teslahunt.io/?country=Euro&modelLetter=y')

# Función para scroll profundo
def scroll_to_bottom(driver, max_attempts=50):
    """
    Realiza un scroll profundo con intentos máximos para cargar todo el contenido.
    """
    last_height = driver.execute_script("return document.body.scrollHeight")
    attempts = 0

    while attempts < max_attempts:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Tiempo suficiente para cargar contenido
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            attempts += 1
        else:
            attempts = 0
        last_height = new_height

    print("Scroll completado.")

# Realizar el scroll profundo
scroll_to_bottom(driver)



wait = WebDriverWait(driver, 10)  # Esperar un máximo de 10 segundos
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

Scroll completado.


In [3]:

coches_data = []
colores_posibles = ['white', 'blue', 'black', 'silver', 'red', 'grey']
paises_interes = ['spain', 'france', 'portugal']
coches_vistos = set()  # Set para evitar duplicados

# Función para encontrar coincidencias parciales en los hashtags
def encontrar_color(hastags_list, colores_posibles):
    for tag in hastags_list:
        for color in colores_posibles:
            # Coincidencia insensible a mayúsculas usando regex
            if re.search(rf'\b{color}\b', tag.lower()):
                return color.capitalize()  # Devolver el color con mayúscula inicial
    return 'Desconocido'


def contiene_pais(hastags_list, paises_interes):
    for tag in hastags_list:
        for pais in paises_interes:
            if pais.lower() in tag.lower():
                return pais.capitalize()
    return None
coches = soup.find_all('div', class_='css-1yz7e9k')

for coche in coches:
    try:
        modelo = coche.find('p', class_='css-1yuuaeh').text.strip()
        datos = coche.find('p', class_='css-iq7v1d').get_text(separator=' ')
        parts = datos.replace('€', '').replace('kms', '').split('•')
        precio = parts[0].strip()
        año = parts[1].strip()
        kilometraje = parts[2].strip()
        
        identificador = (modelo, precio, kilometraje)
        if identificador in coches_vistos:
            continue
        coches_vistos.add(identificador)

        hastags = coche.find('div', class_='css-1ugf06a')
        hastags_list = [i.text.strip() for i in hastags.find_all('button')] if hastags else []
        color = encontrar_color(hastags_list, colores_posibles)
        pais = contiene_pais(hastags_list, paises_interes)

        if pais:
            coche_info = {
                'Modelo': modelo,
                'Precio (€)': precio,
                'Año': año,
                'Kilometraje (kms)': kilometraje,
                'Color': color,
                'País': pais,
                'Hashtags': ', '.join(hastags_list)
            }
            coches_data.append(coche_info)
    
    except Exception as e:
        print("Error procesando coche:", e)

driver.quit()

df_teslahunt = pd.DataFrame(coches_data)


Error procesando coche: 'NoneType' object has no attribute 'text'


In [4]:
df_teslahunt

,Modelo,Precio (€),Año,Kilometraje (kms),Color,País,Hashtags
0,Tesla Model Y Long Range AWD,"38,500",2022,"43,516",Silver,France,"#france, #enhanced autopilot, #standard connec..."
1,Tesla Model Y Standard Range,"40,500",2024,"12,146",Desconocido,Spain,"#spain, #enhanced autopilot, #standard connect..."
2,Tesla Model Y Standard Range,"41,475",2025,0,White,Portugal,"#portugal, #autopilot, #standard connectivity,..."
3,Tesla Model Y Standard Range,"41,475",2024,4,White,Portugal,"#portugal, #autopilot, #standard connectivity,..."
4,Tesla Model Y Standard Range,"42,025",2024,"5,432",White,Portugal,"#portugal, #autopilot, #premium connectivity, ..."
...,...,...,...,...,...,...,...
191,Tesla Model Y Performance,"62,825",2024,9,Blue,Portugal,"#portugal, #carbon fibre spoiler, #autopilot, ..."
192,Tesla Model Y Performance,"62,930",2025,2,Desconocido,France,"#france, #carbon fibre spoiler, #enhanced auto..."
193,Tesla Model Y Performance,"64,815",2024,13,Blue,Portugal,"#portugal, #carbon fibre spoiler, #autopilot, ..."
194,Tesla Model Y Performance,"66,420",2024,10,Red,Spain,"#spain, #carbon fibre spoiler, #enhanced autop..."


In [5]:
# Opcional: Guardar en un archivo CSV
df_teslahunt.to_csv('tesla_coches_Y_prueba.csv', index=False)   

In [19]:
archivos_csv = ['tesla_coches_S_prueba.csv', 'tesla_coches_3_prueba.csv', 'tesla_coches_X_prueba.csv', 'tesla_coches_Y_prueba.csv']


In [20]:
df_modelos_combinados = pd.concat([pd.read_csv(archivo) for archivo in archivos_csv], ignore_index=True)


In [21]:
df_modelos_combinados = df_modelos_combinados.drop_duplicates()
df_modelos_combinados = df_modelos_combinados.drop(columns=['Hashtags'])


In [22]:
# Diccionarios para traducir los colores y países
traduccion_colores = {
    'White': 'Blanco',
    'Blue': 'Azul',
    'Black': 'Negro',
    'Silver': 'Plata',
    'Red': 'Rojo',
    'Grey': 'Gris'
}

traduccion_paises = {
    'Spain': 'España',
    'France': 'Francia',
    'Portugal': 'Portugal'
}

# Traducir los colores
df_modelos_combinados['Color'] = df_modelos_combinados['Color'].map(traduccion_colores).fillna(df_modelos_combinados['Color'])

# Traducir los países
df_modelos_combinados['País'] = df_modelos_combinados['País'].map(traduccion_paises).fillna(df_modelos_combinados['País'])

##  CSV CON TODOS LOS MODELOS DE ESPAÑA FRANCIA Y PORTUGAL

In [23]:
df_modelos_combinados.to_csv('tesla_coches_combinados.csv', index=False)


In [24]:
df_modelos_combinados['País'].value_counts()

País
Francia     256
España      230
Portugal    137
Name: count, dtype: int64